# SMARTS/SMIRKS Highlighter

This notebook was created to help with molecule visualization using substructure searches. 
You provide 
- a list of molecules (demonstrated here with SMILES strings, but you could set it up to read in a file)
- a SMILES/SMARTS/SMIRKS you want to match to the molecule

Then an image is created for every substructure in each molecule matched by that SMILES/SMARTS/SMIRKS.

**Import important modules**

In [16]:
from openeye import oechem
import openeye.oedepict as oedepict
import IPython
from IPython.display import display, Image
from openforcefield.utils import read_molecules

### Function for visualizing molecules with key atoms highlighted

In [17]:
def  depictMatch(mol, match, supH = True, idx_atoms=list(), width=500, height=200, fn=None):
    
    atom_bond_set = oechem.OEAtomBondSet()
    for atom in mol.GetAtoms():
        if atom.GetIdx() in idx_atoms:
            atom_bond_set.AddAtom(atom)
            for bond in atom.GetBonds():
                nbr_atom = bond.GetNbr(atom)
                if (nbr_atom.GetIdx() in idx_atoms) and nbr_atom.GetIdx() > atom.GetIdx():
                    atom_bond_set.AddBond(bond)
    
    dopt = oedepict.OEPrepareDepictionOptions()
    dopt.SetDepictOrientation( oedepict.OEDepictOrientation_Horizontal)
    dopt.SetSuppressHydrogens(supH)
    oedepict.OEPrepareDepiction(mol, dopt)
    
    opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
    opts.SetAtomColorStyle(oedepict.OEAtomColorStyle_WhiteMonochrome)
    disp = oedepict.OE2DMolDisplay(mol, opts)
    
    # Highlight indexed atoms
    hstyle = oedepict.OEHighlightStyle_Cogwheel
    hcolor = oechem.OEColor(oechem.OELightBlue)
    if not atom_bond_set.IsEmpty():
        oedepict.OEAddHighlighting(disp, hcolor, hstyle, atom_bond_set)
    
    # Highlight whole match
    hstyle = oedepict.OEHighlightStyle_BallAndStick
    hcolor = oechem.OEColor(oechem.OELightSalmon)
    oedepict.OEAddHighlighting(disp, hcolor, hstyle, match)
    
    img = oedepict.OEImage(width, height)
    oedepict.OERenderMolecule(img,disp)
    
    if fn is not None:
        ext = oechem.OEGetFileExtension(fn)
        if oedepict.OEIsRegisteredImageFile(ext):
            ofs = oechem.oeofstream()
            if not ofs.open(fn):
                print("OE could not open file due to error %s" % fn)
                ofs.close()
            else:
                oedepict.OERenderMolecule(ofs, ext, disp)
                ofs.close()
        else:
            print("OE could not handle extension on %s" % fn)
    return img

### Make a list of Molecules

In [18]:
# start with a list of SMILES:
smiles = ['CC12CCC3c4ccc(cc4CC(C3C1CCC2O)CCCCCCCCCS(=O)CCCC(C(F)(F)F)(F)F)O']

In [39]:
mols = read_molecules('zinc-subset-parm@frosst.mol2.gz')

Loading molecules from '/Users/bannanc/miniconda/envs/openff/lib/python3.6/site-packages/openforcefield/data/molecules/zinc-subset-parm@frosst.mol2.gz'...
7505 molecules read
1.128 s elapsed


### Assign a SMILES/SMARTS/SMIRKS substructure search

In [40]:
# write a string for your substructure search
subsearch = 'CS(=O)(=O)Nc1ccncn1'

### Do you want to see the hydrogens on your molecules?

In [41]:
see_hydrogens = False

### Do you want to use the MDL aromaticity model?

In [42]:
mdl = False

### Search each molecule for the pattern

In [43]:
unique = True
save_file = True
file_formula = "/Users/bannanc/Desktop/highlight_mols/triv_%i_%i.pdf"
for smi_idx, smi in enumerate(smiles):
#for smi_idx, mol in enumerate(mols):
    #mol = oechem.OEMol()
    #if not oechem.OESmilesToMol(mol, smi):
    #    print("coun't parse %s" % smi)
    #    continue
        
    #oechem.OEAddExplicitHydrogens(mol)
    
    if mdl:
        oechem.OEClearAromaticFlags(mol)
        oechem.OEAssignAromaticFlags(mol, oechem.OEAroModelMDL)
        oechem.OEAssignHybridization(mol)
    
    # make a substructure search object
    qmol = oechem.OEQMol()
    if not oechem.OEParseSmarts(qmol, subsearch):
        print("FAILED to parse")
        break
    ss = oechem.OESubSearch(qmol)
    
    #print(mol.GetTitle(), oechem.OEMolToSmiles(mol), ss.SingleMatch(mol))
    
    for match_idx, match in enumerate(ss.Match(mol, unique)):
        # get indexed atoms (if they exist)
        idx_dict = dict()
        for ma in match.GetAtoms():
            pat = ma.pattern.GetMapIdx()
            if pat > 0:
                idx_dict[pat] = ma.target.GetIdx()
                
        print(idx_dict)
        # print(subsearch)
        idx_atoms = [idx_dict[k] for k in sorted(idx_dict.keys())]
        if save_file:
            fn = file_formula % (smi_idx, match_idx)
        else:
            fn = None
        img = depictMatch(mol,match,supH=not see_hydrogens, idx_atoms=idx_atoms, fn=fn)
        display(Image(oedepict.OEWriteImageToString("png",img)))

In [8]:
img = depictMatch(mol,match,supH=not see_hydrogens, idx_atoms=idx_atoms, fn="/Users/bannanc/Desktop/test1.pdf")

In [ ]:
img.width